# (EX) News article processing (with ML pipeline)

# `agnews` Dataset

In [ ]:
!curl https://raw.githubusercontent.com/mosesyhc/de300-2025sp-class/refs/heads/main/agnews.csv -O

# Pipelining with PySpark MLlib

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline # pipeline to transform data


In [ ]:
spark = (SparkSession.builder
         .master("local[*]")
         .appName("AG news")
         .getOrCreate()
        )
sc = spark.sparkContext

In [ ]:
# load dataset
df = 

# Arrange columns

In [ ]:
from pyspark.sql.functions import concat_ws, col # to concatinate cols

# renaming 'Class Index' col to 'label'

# concatenating texts

df.show(10)

# Tokenize

In [ ]:
from pyspark.ml.feature import RegexTokenizer # tokenizer

# convert sentences to list of words

# Stopwords

In [ ]:
from pyspark.ml.feature import StopWordsRemover

# remove stopwords

# Term frequency, Inverse document frequency

In [ ]:
from pyspark.ml.feature import HashingTF

# calculate term frequency in each article (row)


In [ ]:
from pyspark.ml.feature import IDF

# inverse document frequency

In [ ]:
rescaled_data.select('raw_features').show(2, truncate=False)
rescaled_data.select('features').show(2, truncate=False)

# Training a multinomial logistic regression

In [ ]:
# split data into training and testing


In [ ]:
from pyspark.ml.classification import LogisticRegression


# Prediction and evaluation

In [ ]:
# predict on test data
predictions = lrModel.transform(test)

In [ ]:
from pyspark.sql.functions import avg
from pyspark.sql.types import FloatType

# accuracy calculation

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
# labels = ["World", "Sports", "Business","Science"]

# take only the predictions
preds_and_labels = 


In [ ]:
# confusion matrix
metrics = 

# Pipelining, from start to finish

In [ ]:
# load dataset
df = spark.read.csv("agnews.csv", inferSchema=True, header=True)

def arrangeColumns(df):
  # Renaming 'Class Index' col to 'label'
  df = df.withColumnRenamed('Class Index', 'label')

  # Add a new column 'text' by joining 'Title' and 'Description'
  df = df.withColumn("text", concat_ws(" ", "Title", 'Description'))

  # Select new text feature and labels
  df = df.select('label', 'text')
  return df

df = arrangeColumns(df)

# tokenizer
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

# stopwords
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered")

# term frequency
hashing_tf = HashingTF(inputCol="filtered",
                       outputCol="raw_features",
                       numFeatures=16384)

# Inverse Document Frequency
idf = IDF(inputCol="raw_features", outputCol="features")

# model
lr = LogisticRegression(featuresCol='features',
                        labelCol='label',
                        family="multinomial",
                        regParam=0.3,
                        elasticNetParam=0,
                        maxIter=20)



In [ ]:
# Put everything in pipeline
pipeline = Pipeline(stages=[tokenizer,
                            stopwords_remover,
                            hashing_tf,
                            idf,
                            lr])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df)

# transform and train
dataset = pipelineFit.transform(df)